## Tools

Tools allow agent to perform operations/activities. Proper description helps LLM to use tools intelligently.

LangChain supports many tool formats and tool sets. Check [official tools documentation](https://docs.langchain.com/oss/python/langchain/overview) for complete reference

### Calculator example

The docstring, inferred arguments and argument types are used by LLM to determine when and how to use tools

In [1]:
from typing import Literal
from langchain_core.tools import tool

@tool
def real_number_calculator(a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]) -> float:
    """Performs basic arithmetic operations on two real numbers."""
    print("Invoking real_number_calculator tool...")
    
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Cannot divide by zero.")
        return a / b
    else:
        raise ValueError(f"Unsupported operation: {operation}")

In [17]:
from langchain.agents import create_agent

agent = create_agent(
    # model="ollama:gpt-oss:20b-cloud",
    model="openai:gpt-5-mini",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

Let's invoke calculator tool

In [15]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is 15.5 divides by 3.2?"}]}
)
print(result["messages"][-1].content)

Invoking real_number_calculator tool...
15.5 ÷ 3.2 = 4.84375


Let's try with integer inputs

In [16]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is 15 times 3?"}]}
)

print(result["messages"][-1].content)

15 times 3 is 45.


In [20]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is 15 plus 7.5?"}]}
)
print(result["messages"][-1].content)

15 + 7.5 = 22.5.


The calculator tool may not get invoked. However, it may vary according to LLM, tool docstring, description and prompt

#### Add more detailed description so that model can get better understanding about the tool

While a basic description is often sufficient, Enhanced description helps LLM to understand the tool better. Example below uses Google style argument description method. Used with `parse_docstring=True` will parse and pass arg description to the model. 

In [21]:
from typing import Literal
from langchain.tools import tool

@tool(
    "calculator",
    parse_docstring=True,
    description="""
        Perform basic arithmetic operations on two real numbers.
        Use this whenever you need to add, subtract, multiply, or divide real numbers (including decimals and integers).
    """
)
def real_number_calculator(
    a: float,
    b: float,
    operation: Literal["add", "subtract", "multiply", "divide"],
) -> float:
    """Performs basic arithmetic operations on two real numbers.
    
    Args:
        a (float): The first real number.
        b (float): The second real number.
        operation (Literal["add", "subtract", "multiply", "divide"]): 
        
        The arithmetic operation to perform:

            - "add": Returns the sum of a and b.
            - "subtract": Returns the difference when b is subtracted from a.
            - "multiply": Returns the product of a and b.
            - "divide": Returns the quotient when a is divided by b. Raises an error if b is zero.

    Returns:
        float: The result of the arithmetic operation.

    Raises:
        ValueError: If an unsupported operation is provided or if division by zero is attempted.

    """

    print("Invoking real_number_calculator tool...")
    
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Cannot divide by zero.")
        return a / b
    else:
        raise ValueError(f"Unsupported operation: {operation}"
)

In [22]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

In [27]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "a = 5; y = 3a + 2. What is the value of y?"}]}
)

print(result["messages"][-1].content)

Invoking real_number_calculator tool...
Invoking real_number_calculator tool...
y = 3·5 + 2 = 15 + 2 = 17.
